# Relaciones

In [22]:
#conexiones a aura
from neo4j import GraphDatabase
uri = "neo4j+ssc://5fb4d30f.databases.neo4j.io"
auth= ("neo4j", "8zCnpPpgQmkE8vEsazT7nPT8y4DWHvZ9dq0aPaD5qVc")


In [19]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [23]:
def crear_relaciones(uri, auth, relationship_type, properties_keys):

    empty_properties = {key: "" if isinstance(key, str) else 0 for key in properties_keys} #cre

    query = f"""
    MATCH ()-[r:{relationship_type}]->()
    SET r += $empty_properties
    RETURN COUNT(r) AS total_updated
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            result = session.run(query, empty_properties=empty_properties)
            return result.single()["total_updated"]

In [24]:
# Propiedades para ACTED_IN
properties_keys = ["Role", "ScreenTime", "CharacterImportance"]
relacion = "ACTED_IN"
total = crear_relaciones(uri, auth, relacion, properties_keys)


Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías


In [25]:
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 62841 relaciones ACTED_IN con nuevas propiedades vacías


In [26]:
#propiedades para DIRECTED
properties_keys = ["Year", "AwardNominated", "FilmFestival"]
relacion = "DIRECTED"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 4669 relaciones DIRECTED con nuevas propiedades vacías


In [27]:
#Prpiedades para directed by
properties_keys = ["Year", "AwardNominated", "FilmFestival"]
relacion = "DIRECTED_BY"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 4669 relaciones DIRECTED_BY con nuevas propiedades vacías


In [29]:
#Prpiedades para FILMED_IN
properties_keys = ["City", "Days", "Year"]
relacion = "FILMED_IN"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 49350 relaciones FILMED_IN con nuevas propiedades vacías


In [30]:
#Prpiedades para IS FRIENDS WITH
properties_keys = ["Since", "CommonInterests", "FrequenciaInteraccion"]
relacion = "IS_FRIENDS_WITH"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 2026 relaciones IS_FRIENDS_WITH con nuevas propiedades vacías


In [31]:
#Prpiedades para IS TYPE
properties_keys = ["CommunityRating", "ReviewText", "MatchAccuracy"]
relacion = "IS_TYPE"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 91015 relaciones IS_TYPE con nuevas propiedades vacías


In [32]:
#Prpiedades para OPERATS IN
properties_keys = ["Marketshare", "EmployeeCount", "YearsEstablished"]
relacion = "OPERATES_IN"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 34853 relaciones OPERATES_IN con nuevas propiedades vacías


In [33]:
#Prpiedades para PRODUCED BY
properties_keys = ["Budget", "BoxOffice", "Reception"]
relacion = "PRODUCED_BY"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 4 relaciones PRODUCED_BY con nuevas propiedades vacías


In [34]:
#Prpiedades para RATED
properties_keys = ["Ratingvalue", "RatingDate", "ReviewText"]
relacion = "RATED"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 2514 relaciones RATED con nuevas propiedades vacías


In [35]:
#Prpiedades para RECOMMENDED_BY
properties_keys = ["Ratingvalue", "RatingDate", "ReviewText"]
relacion = "WAS_RECOMMENDED_BY"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 2514 relaciones WAS_RECOMMENDED_BY con nuevas propiedades vacías


## Recomendador

In [53]:
driver = GraphDatabase.driver(uri, auth=auth)

In [59]:
#verificar que exista el usuario
def user_exists(user_id):
    """
    Retorna True si existe un nodo User con la propiedad user_id = user_id,
    False en caso contrario.
    """
    query = """
    MATCH (u:User {UserID: $user_id})
    RETURN count(u) AS count
    """
    with driver.session() as session:
        result = session.run(query, user_id=user_id)
        record = result.single()
        if record and record["count"] > 0:
            return True
        return False

In [64]:
def recommend_by_user(user_id, rating_threshold=3.5, top_n=10):
    """
    Recomienda películas que el usuario no haya visto, pero que sus amigos
    hayan valorado por encima de rating_threshold.
    """
    # Primero, verificamos si el usuario existe
    if not user_exists(user_id):
        print(f"Error: No se encontró ningún usuario con ID = {user_id}.")
        return []
    
    query = """
    MATCH (u:User {UserID: $user_id})-[:IS_FRIENDS_WITH]->(friend:User)-[r:RATED]->(m:Movie)
    WHERE NOT (u)-[:RATED]->(m)
      AND r.Rating >= $rating_threshold
    RETURN m.Title AS title, avg(r.Rating) AS avg_rating
    ORDER BY avg_rating DESC
    LIMIT $top_n
    """
    with driver.session() as session:
        result = session.run(
            query,
            user_id=user_id,
            rating_threshold=rating_threshold,
            top_n=top_n
        )
        return [record.data() for record in result]


In [81]:
# personalizado
def search_movies(
    genre=None,
    year=None,
    duration=None,
    keywords=None,
    company=None,
    director=None,
    top_n=10
):
    """
    Devuelve películas que cumplan con los criterios dados:
      - genre: nombre del género (string)
      - year: año ± 2 (int), extraído de la propiedad datetime 'Released'
      - duration: duración ± 30 (int)
      - keywords: búsqueda parcial en la propiedad 'Keywords'
      - company: búsqueda parcial en el nombre de la compañía
      - director: búsqueda parcial en el nombre del director
    """
    base_query = """
    MATCH (m:Movie)
    OPTIONAL MATCH (m)-[:IS_TYPE]->(g:Genre)
    OPTIONAL MATCH (m)-[:PRODUCED_BY]->(c:Company)
    OPTIONAL MATCH (m)<-[:DIRECTED]-(d:Director)
    WHERE 1=1
    """
    conditions = []
    
    # 1) Filtro por género exacto
    if genre:
        conditions.append("g.Name = $genre")
    
    # 2) Filtro por año ± 2 utilizando la propiedad datetime 'Released'
    if year:
        conditions.append("(m.Released.year >= $yearMin AND m.Released.year <= $yearMax)")
    
    # 3) Filtro por duración ± 30
    if duration:
        conditions.append("(m.Runtime >= $durationMin AND m.Runtime <= $durationMax)")
    
    # 4) Filtro por keywords en la propiedad 'Keywords'
    if keywords:
        conditions.append("toLower(m.Keyword) CONTAINS toLower($keywords)")
    
    # 5) Filtro por compañía (búsqueda parcial)
    if company:
        conditions.append("toLower(c.Name) CONTAINS toLower($company)")
    
    # 6) Filtro por director (búsqueda parcial)
    if director:
        conditions.append("toLower(d.Name) CONTAINS toLower($director)")
    
    if conditions:
        base_query += " AND " + " AND ".join(conditions)
    
    base_query += """
    RETURN DISTINCT m.Title AS title, m.Released.year AS year, m.Runtime AS duration
    ORDER BY year DESC
    LIMIT $top_n
    """
    
    # Parámetros para la consulta
    params = {
        "genre": genre,
        "keywords": keywords,
        "company": company,
        "director": director,
        "top_n": top_n
    }
    if year:
        params["yearMin"] = year - 2
        params["yearMax"] = year + 2
    if duration:
        params["durationMin"] = duration - 30
        params["durationMax"] = duration + 30
    
    with driver.session() as session:
        result = session.run(base_query, **params)
        return [record.data() for record in result]


In [71]:
def main():
    print("Bienvenido al sistema de recomendación de películas.")
    print("1) Recomendación por usuario (basada en amigos).")
    print("2) Búsqueda personalizada (género, año ± 2, duración ± 30, palabras clave, compañía, director).")
    
    opcion = input("Elige una opción (1/2): ").strip()
    
    if opcion == "1":
        # Recomendación por usuario
        user_id_str = input("Ingresa el ID del usuario: ").strip()
        if not user_id_str.isdigit():
            print("Error: El ID de usuario debe ser un número entero.")
            return
        user_id = int(user_id_str)
        
        # Se recomienda con threshold de rating y top_n fijos, puedes modificar si quieres
        recomendaciones = recommend_by_user(user_id, rating_threshold=3.5, top_n=10)
        
        if recomendaciones:
            print(f"\n=== Recomendaciones para el usuario {user_id} (rating >= 3.5) ===")
            for rec in recomendaciones:
                print(f"{rec['title']} (Valoración promedio: {rec['avg_rating']:.2f})")
        else:
            print("No se encontraron recomendaciones para este usuario o el usuario no existe.")
    
    elif opcion == "2":
        # Búsqueda personalizada
        print("Puedes dejar cualquier campo en blanco para omitirlo.")
        
        genre = input("Género: ").strip()
        year_input = input("Año (± 2): ").strip()
        duration_input = input("Duración aproximada (en minutos): ").strip()
        keywords = input("Palabras clave (en la columna 'Keywords'): ").strip()
        company = input("Compañía/Estudio : ").strip()
        director = input("Director : ").strip()
        
        # Validamos si year y duration son dígitos
        year = None
        if year_input:
            if year_input.isdigit():
                year = int(year_input)
            else:
                print("Aviso: El año ingresado no es un número. Se ignorará el filtro de año.")
        
        duration = None
        if duration_input:
            if duration_input.isdigit():
                duration = int(duration_input)
            else:
                print("Aviso: La duración ingresada no es un número. Se ignorará el filtro de duración.")
        
        resultados = search_movies(
            genre=genre if genre else None,
            year=year,
            duration=duration,
            keywords=keywords if keywords else None,
            company=company if company else None,
            director=director if director else None,
            top_n=10
        )
        
        if resultados:
            print("\n=== Resultados de búsqueda ===")
            for r in resultados:
                print(f"{r['title']} (Año: {r['year']}, Duración: {r['duration']} min)")
        else:
            print("No se encontraron películas que cumplan los criterios (o no existen en la BD).")
    
    else:
        print("Opción no válida.")


In [83]:
if __name__ == "__main__":
    try:
        driver = GraphDatabase.driver(uri, auth=auth)
        main()
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        driver.close()

Bienvenido al sistema de recomendación de películas.
1) Recomendación por usuario (basada en amigos).
2) Búsqueda personalizada (género, año ± 2, duración ± 30, palabras clave, compañía, director).


Elige una opción (1/2):  2


Puedes dejar cualquier campo en blanco para omitirlo.


Género:  Comedy
Año (± 2):  1995
Duración aproximada (en minutos):  90
Palabras clave (en la columna 'Keywords'):  Friends, toys
Compañía/Estudio :  Pixar
Director :  



=== Resultados de búsqueda ===
War Stories Our Mother Never Told Us (Año: None, Duración: 95.0 min)
Vermont Is for Lovers (Año: None, Duración: 88.0 min)
Jails, Hospitals & Hip-Hop (Año: None, Duración: 90.0 min)
Boricua's Bond (Año: None, Duración: 105.0 min)
Divine Intervention (Año: None, Duración: 87.0 min)
Bling: A Planet Rock (Año: None, Duración: 90.0 min)
Dead Ahead: The Exxon Valdez Disaster (Año: None, Duración: 0.0 min)
Das Experiment 770 - Gebären auf Befehl (Año: None, Duración: 52.0 min)
Connections (Año: None, Duración: 0.0 min)
Human Failure (Año: None, Duración: 91.0 min)


In [43]:
import datetime
def menu():
    print("BIENVENIDO AL RECOMENDADOR DE PELICULAS")
    print("Ingresa tus gustos y preferencias a continuación:")


def validar_fecha(fecha_texto):
    try:
        datetime.datetime.strptime(fecha_texto, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def procesar_entrada_lista(entrada, capitalizar=False):
    lista = entrada.split(',')
    if capitalizar:
        # Capitalizar la primera letra de cada palabra en cada elemento de la lista.
        return [item.strip().title() for item in lista]
    else:
        return [item.strip() for item in lista]

def variables_recomendacion():
    genero = procesar_entrada_lista(input("Ingresa los géneros que te gustan, separados por comas: "), True)
    
    while True:
        released = input("Ingresa la fecha de estreno que prefieres (formato YYYY-MM-DD): ").strip()
        if validar_fecha(released):
            break
        print("Formato de fecha incorrecto, por favor intenta de nuevo.")
    
    while True:
        duracion = input("Ingresa la duración mínima que prefieres para la película (en minutos): ").strip()
        if duracion.isdigit():
            duracion = int(duracion)
            break
        print("Por favor ingresa un número válido.")
    
    keywords = procesar_entrada_lista(input("Ingresa palabras clave relacionadas con la película, separadas por comas: "))
    studio = input("Ingresa el estudio de cine que prefieres: ").strip().title()
    director = input("Ingresa el nombre del director que te gustaría ver: ").strip().title()
    actors = procesar_entrada_lista(input("Ingresa los nombres de los actores que te gustan, separados por comas: "), True)

    return genero, released, duracion, keywords, studio, director, actors


#############################################

def neo4j_connect(uri, auth):
    driver = GraphDatabase.driver(uri, auth=auth)
    return driver

def get_movie_recommendations(driver, genero, released, duracion, keywords, studio, director, actors):
    # Ajustar el rango de fecha
    year = int(released.split("-")[0])
    min_year = year - 5
    max_year = year + 5
    
    # Ajustar el rango de duración
    min_runtime = duracion - 30
    max_runtime = duracion + 30
    
    query = """
MATCH (m:Movie)<-[:ACTED_IN]-(a:Actor), 
      (m)-[:IS_TYPE]->(g:Genre), 
      (m)<-[:DIRECTED_BY]-(d:Director), 
      (m)<-[:PRODUCED_BY]-(c:Company)
WHERE 
    g.Name IN $genres AND
    m.Released >= $minYear AND m.Released <= $maxYear AND
    m.Runtime >= $minRuntime AND m.Runtime <= $maxRuntime AND
    any(keyword IN m.Keywords WHERE keyword IN $keywords) AND
    c.Name = $studio AND
    d.Name = $director 
WITH m, collect(a.Name) AS actorNames
WHERE any(actor IN $actors WHERE actor IN actorNames)
RETURN m.Title AS Title, m.Released AS Released, m.Runtime AS Runtime
LIMIT 10
"""

    
    with driver.session() as session:
        result = session.run(query, {
            'genres': genero,
            'minYear': min_year,
            'maxYear': max_year,
            'minRuntime': min_runtime,
            'maxRuntime': max_runtime,
            'keywords': keywords,
            'studio': studio,
            'director': director,
            'actors': actors
        })
        return list(result)

# Conexión a Neo4j

driver = neo4j_connect(uri, auth)

################### MAIN

menu()

# Recoger inputs del usuario
inputs = variables_recomendacion()
genero, released, duracion, keywords, studio, director, actors = inputs

# Obtener recomendaciones
recommendations = get_movie_recommendations(driver, *inputs)

#Imprimir recomendaciones
for movie in recommendations:
    print(movie['Title'], movie['Released'], movie['Runtime'])
############################################



BIENVENIDO AL RECOMENDADOR DE PELICULAS
Ingresa tus gustos y preferencias a continuación:


Ingresa los géneros que te gustan, separados por comas:  comedy
Ingresa la fecha de estreno que prefieres (formato YYYY-MM-DD):  1995-10-30
Ingresa la duración mínima que prefieres para la película (en minutos):  90
Ingresa palabras clave relacionadas con la película, separadas por comas:  friends, toys
Ingresa el estudio de cine que prefieres:  Pixar
Ingresa el nombre del director que te gustaría ver:  
Ingresa los nombres de los actores que te gustan, separados por comas:  


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: Keywords)} {position: line: 10, column: 22, offset: 329} for query: '\nMATCH (m:Movie)<-[:ACTED_IN]-(a:Actor), \n      (m)-[:IS_TYPE]->(g:Genre), \n      (m)<-[:DIRECTED_BY]-(d:Director), \n      (m)<-[:PRODUCED_BY]-(c:Company)\nWHERE \n    g.Name IN $genres AND\n    m.Released >= $minYear AND m.Released <= $maxYear AND\n    m.Runtime >= $minRuntime AND m.Runtime <= $maxRuntime AND\n    any(keyword IN m.Keywords WHERE keyword IN $keywords) AND\n    c.Name = $studio AND\n    d.Name = $director \nWITH m, collect(a.Name) AS actorNames\nWHERE any(actor IN $